Here we will explore collecting songs from spotify APIs and how to ask for what we want, that is:
- Once the user inputs a song, we will ask spotify for the features of the song
- Then we will match with another song with similar features, and recommend it to the user
The steps are:
- Find playlists in spotify with lots of songs
- Create a list with them
- Extract name of songa and audio features from each one
- Create a DF

Then:
- Cluster songs according to audio features

In this notebook I will work with playlists containing female artists only.

In [93]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [94]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="6582a8f807004727b3bc8a7c281ae02c",
    client_secret="c38fc3efa7f74def856cb96d000fd6ea"))

In [95]:
#create a list of playlists url's:

list_url = ["https://open.spotify.com/playlist/13qezNPeoOZbnpgYdvfXzw", "https://open.spotify.com/playlist/7vse1mpYM4ZZMAnFjtDmxn", "https://open.spotify.com/playlist/5BrB90Tlv372JYksHKEUUC", "https://open.spotify.com/playlist/4gGVp5hZBGMT5u8Umwy71i", "https://open.spotify.com/playlist/69IzgRxJudWrroRkFz6rqD", "https://open.spotify.com/playlist/7Bz2d7LtQAA4sdqjK54ihN", "https://open.spotify.com/playlist/3jN4ozUr5QnxLJAuWSOz8L", "https://open.spotify.com/playlist/7nb8QoKz6X7j43a8JLDWWa", "https://open.spotify.com/playlist/0Elk90H3u3Pii6HmU7uwDQ", "https://open.spotify.com/playlist/5jL36juHp2hI5p4w7Otkp8", "https://open.spotify.com/playlist/1FNxkddOTy0dEQjzO605Mu", "https://open.spotify.com/playlist/71AoD5gr7igctQEioLcrgs", "https://open.spotify.com/playlist/28PCyERc69PVABnVlhzCzK", "https://open.spotify.com/playlist/1AHXVPpZj0ttq2kEHfRrBz", "https://open.spotify.com/playlist/7GQpGQ7mo1qNgjRYzTIROw", "https://open.spotify.com/playlist/1n6ApCcBNm1LlObavhViZV", "https://open.spotify.com/playlist/52p0TeTGoMt2bdYaCgDgGi", "https://open.spotify.com/playlist/1vs8WUfA0QQO4V0rtVpFKX"]


In [96]:
#with "next"
playlists = []
for i in list_url:
    track = sp.user_playlist_tracks("spotify", i)
    playlists.append(track)
    while track['next']:
        track = sp.next(track)
        playlists.append(track) 
print(len(playlists))


37


In [97]:
len(playlists[0]["items"])

100

In [98]:
playlists[0].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [99]:
playlists[0]['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [100]:
playlists[0]['items'][0]['track']['artists'][0]["name"]

'Evanescence'

In [101]:
## making if iterate trhough the list of playlists I have:
songs = []
for i in range(len(playlists)):
    for song in playlists[i]["items"]:
        songs.append(song["track"]["name"])
print(songs)

['Bring Me To Life', 'Like The Way I Do', 'The Chain - 2004 Remaster', 'Stars', 'White Rabbit', 'Total Eclipse of the Heart', "Your Daddy Don't Know", 'What About Love?', 'You Keep Me Hangin On', 'Time After Time', 'Call Me', "Don't Speak", "What's Up?", 'Crimson and Clover', 'Back on the Chain Gang - 2007 Remaster', 'Love Is A Battlefield', 'Celebrity Skin', 'Whatcha Do to My Body', 'Tell It to My Heart', 'Running Up That Hill (A Deal With God) - 2018 Remaster', 'Complicated', "Don't It Make Ya Feel", 'Close My Eyes Forever', 'High School Confidential', 'The Ballad Of Lucy Jordan', 'Big Yellow Taxi', 'Blackbird', 'Ironic - 2015 Remaster', 'What I Am', 'Me and Bobby McGee', 'I Just Want to Make Love to You', 'Something To Talk About', 'Barracuda', 'Edge of Seventeen - 2016 Remaster', "Call Me When You're Sober", 'A Whiter Shade of Pale', 'Luka', 'We Belong', 'Bette Davis Eyes', 'Fast Car', 'All I Wanna Do', 'Stupid Girl', 'Brass in Pocket - 2006 Remaster', 'Love Is', '99 Luftballons', 

In [102]:
len(songs)

2975

In [103]:
len(set(songs))

2618

In [104]:
## now, get audio features of the songs:
##try with one song first: search for it, get irs uri, and use in the audio feature function

## sp.search('High Hopes')["tracks"]["items"][0]["uri"]

In [105]:
##now try to iterate create a list with all of the uris
uris = []
for i in range(len(playlists)):
    for uri in playlists[i]["items"]:
        uris.append(uri["track"]["uri"])
print(len(uris))

2975


In [106]:
##now try to iterate create a list with all of the artists
artists = []
for i in range(len(playlists)):
    for artist in playlists[i]["items"]:
        artists.append(artist["track"]["artists"][0]["name"])
print(len(artists))

2975


In [107]:
print(artists)

['Evanescence', 'Melissa Etheridge', 'Fleetwood Mac', 'Grace Potter & The Nocturnals', 'Jefferson Airplane', 'Bonnie Tyler', 'Toronto', 'Heart', 'Kim Wilde', 'Cyndi Lauper', 'Blondie', 'No Doubt', '4 Non Blondes', 'Joan Jett & The Blackhearts', 'Pretenders', 'Pat Benatar', 'Hole', 'Lee Aaron', 'Taylor Dayne', 'Kate Bush', 'Avril Lavigne', 'Headpins', 'Lita Ford', 'Rough Trade', 'Marianne Faithfull', 'Joni Mitchell', 'Sarah McLachlan', 'Alanis Morissette', 'Edie Brickell & New Bohemians', 'Janis Joplin', 'Etta James', 'Bonnie Raitt', 'Heart', 'Stevie Nicks', 'Evanescence', 'Annie Lennox', 'Suzanne Vega', 'Pat Benatar', 'Kim Carnes', 'Tracy Chapman', 'Sheryl Crow', 'Ellen Foley', 'Pretenders', 'Alannah Myles', 'Nena', 'Juice Newton', 'Orianthi', 'Patti Smith', 'Tina Turner', 'Linda Ronstadt', 'Carly Simon', 'Suzi Quatro', 'Heart', 'Heart', 'Melissa Etheridge', 'Joan Jett & The Blackhearts', 'Lita Ford', 'Sheryl Crow', 'Tracy Chapman', 'Meredith Brooks', 'Pat Benatar', 'Natalie Imbruglia'

In [108]:
## get audio features
audio_feat = []
for i in uris:
    audio_feat.append(sp.audio_features(i))
print(len(audio_feat))

2975


In [109]:
audio_feat[2]

[{'danceability': 0.545,
  'energy': 0.67,
  'key': 9,
  'loudness': -8.81,
  'mode': 1,
  'speechiness': 0.0496,
  'acousticness': 0.009,
  'instrumentalness': 0.000822,
  'liveness': 0.0451,
  'valence': 0.481,
  'tempo': 151.553,
  'type': 'audio_features',
  'id': '5e9TFTbltYBg2xThimr0rU',
  'uri': 'spotify:track:5e9TFTbltYBg2xThimr0rU',
  'track_href': 'https://api.spotify.com/v1/tracks/5e9TFTbltYBg2xThimr0rU',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5e9TFTbltYBg2xThimr0rU',
  'duration_ms': 270213,
  'time_signature': 4}]

In [110]:
# now i have: a list with song names, and a list with their audio features. Lets put them in a df

In [111]:
# first, lets separate each feature, so the df has one column to each feature

danceability = []
for song in audio_feat:
    for i in song:
        danceability.append(i["danceability"])
len(danceability)

2975

In [112]:
loudness = []
for song in audio_feat:
    for i in song:
        loudness.append(i["loudness"])
len(loudness)

2975

In [113]:
speechiness = []
for song in audio_feat:
    for i in song:
        speechiness.append(i["speechiness"])
len(speechiness)

2975

In [114]:
acousticness = []
for song in audio_feat:
    for i in song:
        acousticness.append(i["acousticness"])
len(acousticness)

2975

In [115]:
energy = []
for song in audio_feat:
    for i in song:
        energy.append(i["energy"])
len(energy)

2975

In [116]:
instrumentalness = []
for song in audio_feat:
    for i in song:
        instrumentalness.append(i["instrumentalness"])
len(instrumentalness)

2975

In [117]:
tempo = []
for song in audio_feat:
    for i in song:
        tempo.append(i["tempo"])
len(tempo)

2975

In [118]:
import pandas as pd

songs_features_dict = {
    "name_of_song" : songs,
    "artist": artists,
    "danceability": danceability,
    "energy": energy,
    "loudness": loudness,
    "speechiness": speechiness,
    "acousticness": acousticness,
    "instrumentalness": instrumentalness,
    "tempo": tempo,
    "uri": uris
}

songs_features = pd.DataFrame(songs_features_dict)

In [119]:
len(songs) == len(danceability) == len(artists) == len(uris) == len(loudness) == len(speechiness) == len(acousticness) == len(instrumentalness) == len(tempo)

True

In [120]:
songs_features.head()

,name_of_song,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,uri
0,Bring Me To Life,Evanescence,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,spotify:track:0COqiPhxzoWICwFCS4eZcp
1,Like The Way I Do,Melissa Etheridge,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,spotify:track:5cj6toDqMzkbpgkGoRav0S
2,The Chain - 2004 Remaster,Fleetwood Mac,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,spotify:track:5e9TFTbltYBg2xThimr0rU
3,Stars,Grace Potter & The Nocturnals,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,spotify:track:6s7sq2Quh4TxHRfdk6eFxk
4,White Rabbit,Jefferson Airplane,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,spotify:track:4vpeKl0vMGdAXpZiQB2Dtd


In [121]:
songs_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2975 entries, 0 to 2974
Data columns (total 10 columns):
name_of_song        2975 non-null object
artist              2975 non-null object
danceability        2975 non-null float64
energy              2975 non-null float64
loudness            2975 non-null float64
speechiness         2975 non-null float64
acousticness        2975 non-null float64
instrumentalness    2975 non-null float64
tempo               2975 non-null float64
uri                 2975 non-null object
dtypes: float64(7), object(3)
memory usage: 232.5+ KB


Now, we will work on clustering the songs

In [122]:
from sklearn.preprocessing import StandardScaler

In [123]:
list_audio_feat = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "tempo"]
X = songs_features[list_audio_feat]

In [124]:
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo
0,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612
1,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426
2,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553
3,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262
4,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798


In [125]:
X.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo
count,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000,2975.000000
mean,0.597186,0.584039,-7.465878,0.078653,0.305120,0.025088,120.271594
std,0.156572,0.209242,3.106721,0.082144,0.303284,0.117394,29.037916
min,0.138000,0.016700,-26.201000,0.023100,0.000009,0.000000,45.780000
25%,0.495500,0.436000,-9.294000,0.032650,0.040900,0.000000,97.040000
50%,0.608000,0.600000,-6.955000,0.043900,0.195000,0.000003,119.964000
75%,0.707000,0.739000,-5.207500,0.085200,0.535000,0.000204,139.382500
max,0.965000,0.989000,-0.341000,0.746000,0.992000,0.958000,208.282000


In [126]:
scaler = StandardScaler()

In [127]:
X_scaled = scaler.fit_transform(X)

In [128]:
# now I have an array of the features of each song, fitted and transformed, meaning they had been changed according to mean and std deviation
X_scaled

array([[-1.70037397,  1.7158197 ,  1.37720668, ..., -0.98244649,
        -0.21372961, -0.8838068 ],
       [ 0.05630165,  1.63456045,  0.01219432, ..., -0.91916183,
        -0.21373233, -0.64910864],
       [-0.33336094,  0.41089175, -0.43272244, ..., -0.97654344,
        -0.20674391,  1.07744179],
       ...,
       [ 0.09462912,  1.8018589 ,  1.41261973, ..., -0.65335963,
        -0.21339188,  0.33070623],
       [ 0.06907748,  0.84108778,  0.51988877, ..., -0.96368205,
        -0.21374716,  1.65120198],
       [ 0.98893671, -0.66937827, -0.11690237, ...,  1.60562927,
        -0.21374716, -0.42212583]])

In [129]:
#we use Kmeans to cluster
from sklearn.cluster import KMeans

In [130]:
#initializing
my_kmeans = KMeans(random_state=1234)

In [131]:
#store information from my model inside this kmeans
my_kmeans.fit(X_scaled)
#these are the default parameters used:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=1234, tol=0.0001, verbose=0)

In [132]:
# Predicting the clusters. It will gives us an array of clusters:
clusters = my_kmeans.predict(X_scaled)
clusters

array([3, 3, 6, ..., 2, 2, 4])

In [133]:
import pandas as pd
#I want to analyze and explore them in a DF because it is easier
clusters = pd.Series(clusters)
clusters

0       3
1       3
2       6
3       6
4       0
       ..
2970    3
2971    2
2972    2
2973    2
2974    4
Length: 2975, dtype: int32

In [134]:
clusters.value_counts().sort_values()

5     76
2    273
0    334
7    341
6    403
3    454
4    455
1    639
dtype: int64

In [135]:
# I will all a column in my audio features (X) dataframe telling me in which cluster each song belongs
X["cluster"] = clusters

C:\Users\Mariana\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [136]:
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster
0,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,3
1,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,3
2,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,6
3,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,6
4,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,0


In [137]:
# grouping by cluster, I can see better if I identify patterns in the clusters
X.groupby(["cluster"]).mean()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo
cluster,,,,,,,
0,0.458168,0.334979,-10.336042,0.043826,0.717033,0.011962,88.371383
1,0.709814,0.777903,-4.747567,0.069067,0.091180,0.009776,123.790876
2,0.713103,0.649766,-6.473099,0.290729,0.177728,0.006778,125.660055
3,0.575870,0.674414,-6.028407,0.055179,0.169062,0.003884,92.349520
4,0.696077,0.512400,-9.173220,0.057987,0.281191,0.007819,115.061565
5,0.616329,0.597172,-8.522382,0.094475,0.232826,0.695592,126.506066
6,0.466687,0.659186,-6.327556,0.055152,0.185705,0.003977,155.762258
7,0.475886,0.295609,-11.288798,0.044015,0.774868,0.008084,141.401880


In [138]:
## ideally we explore clusters, but let's move on for now.
## let's work with getting a song from user, requestiong its audio features to spotify, and assigning it to a cluster
input_song = 'Time After Time'

In [139]:
result = sp.search(input_song)

In [140]:
result

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time+After+Time&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2BTZIqw0ntH9MvilQ3ewNY'},
       'href': 'https://api.spotify.com/v1/artists/2BTZIqw0ntH9MvilQ3ewNY',
       'id': '2BTZIqw0ntH9MvilQ3ewNY',
       'name': 'Cyndi Lauper',
       'type': 'artist',
       'uri': 'spotify:artist:2BTZIqw0ntH9MvilQ3ewNY'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',

In [141]:
## I extract the URI, because that is what I'll use to get the audio features
song_uri = result["tracks"]["items"][0]["uri"]

In [142]:
## Now I get the audio features from that song
song_af = sp.audio_features(song_uri)

In [143]:
song_af

[{'danceability': 0.726,
  'energy': 0.449,
  'key': 0,
  'loudness': -9.206,
  'mode': 1,
  'speechiness': 0.0286,
  'acousticness': 0.487,
  'instrumentalness': 1.34e-06,
  'liveness': 0.0824,
  'valence': 0.294,
  'tempo': 130.388,
  'type': 'audio_features',
  'id': '7o9uu2GDtVDr9nsR7ZRN73',
  'uri': 'spotify:track:7o9uu2GDtVDr9nsR7ZRN73',
  'track_href': 'https://api.spotify.com/v1/tracks/7o9uu2GDtVDr9nsR7ZRN73',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7o9uu2GDtVDr9nsR7ZRN73',
  'duration_ms': 241333,
  'time_signature': 4}]

In [144]:
song_af_dict = song_af[0]
song_af_dict

{'danceability': 0.726,
 'energy': 0.449,
 'key': 0,
 'loudness': -9.206,
 'mode': 1,
 'speechiness': 0.0286,
 'acousticness': 0.487,
 'instrumentalness': 1.34e-06,
 'liveness': 0.0824,
 'valence': 0.294,
 'tempo': 130.388,
 'type': 'audio_features',
 'id': '7o9uu2GDtVDr9nsR7ZRN73',
 'uri': 'spotify:track:7o9uu2GDtVDr9nsR7ZRN73',
 'track_href': 'https://api.spotify.com/v1/tracks/7o9uu2GDtVDr9nsR7ZRN73',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7o9uu2GDtVDr9nsR7ZRN73',
 'duration_ms': 241333,
 'time_signature': 4}

In [145]:
song_df = pd.DataFrame(song_af_dict,index=[0])
song_df.drop(columns=["key", "mode", "liveness", "valence", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"], inplace=True)
song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo
0,0.726,0.449,-9.206,0.0286,0.487,0.000001,130.388


In [146]:
# Scale (fit and transform), and predict!
song_df_scaled = scaler.fit_transform(song_df)

In [147]:
my_kmeans.predict(song_df_scaled)

array([4])

In [148]:
song_df["cluster"] = my_kmeans.predict(song_df_scaled)

In [149]:
song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster
0,0.726,0.449,-9.206,0.0286,0.487,0.000001,130.388,4


In [150]:
# I need to have now data frames with audio feature transformed, clusters AND name of song:
song_df["name_of_song"] = input_song
song_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster,name_of_song
0,0.726,0.449,-9.206,0.0286,0.487,0.000001,130.388,4,Time After Time


In [151]:
X["name_of_song"] = songs
X["artist"] = artists

C:\Users\Mariana\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster,name_of_song,artist
0,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,3,Bring Me To Life,Evanescence
1,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,3,Like The Way I Do,Melissa Etheridge
2,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,6,The Chain - 2004 Remaster,Fleetwood Mac
3,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,6,Stars,Grace Potter & The Nocturnals
4,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,0,White Rabbit,Jefferson Airplane


In [169]:
cluster_to_lookfor = song_df["cluster"][0]
same_cluster_df = X.loc[X['cluster'] == cluster_to_lookfor]
same_cluster_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster,name_of_song,artist
8,0.686,0.682,-12.154,0.0330,0.0806,0.000155,123.440,4,You Keep Me Hangin On,Kim Wilde
9,0.726,0.449,-9.206,0.0286,0.4870,0.000001,130.388,4,Time After Time,Cyndi Lauper
12,0.565,0.564,-10.044,0.0292,0.1610,0.000000,134.355,4,What's Up?,4 Non Blondes
15,0.669,0.633,-11.903,0.0490,0.0137,0.017800,90.626,4,Love Is A Battlefield,Pat Benatar
19,0.625,0.533,-11.903,0.0596,0.6590,0.002660,108.296,4,Running Up That Hill (A Deal With God) - 2018 ...,Kate Bush
...,...,...,...,...,...,...,...,...,...,...
2941,0.697,0.629,-5.757,0.0425,0.4090,0.000004,123.279,4,Promise,Ciara
2953,0.632,0.723,-9.323,0.0682,0.0166,0.000037,137.958,4,Kisses Down Low,Kelly Rowland
2958,0.730,0.498,-8.629,0.0640,0.1380,0.083000,117.975,4,Bare Wit Me,Teyana Taylor
2962,0.768,0.441,-7.976,0.1430,0.5150,0.000000,132.024,4,Forfeit. (ft. Lucky Daye),Kiana Ledé


In [185]:
choice = same_cluster_df.sample()
choice

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster,name_of_song,artist
221,0.941,0.651,-8.179,0.109,0.215,0.000012,109.908,4,Pretty Ugly,Tierra Whack


In [200]:
print("Try listening to ", choice.iloc[0,8], "by ", choice.iloc[0,9])

Try listening to  Pretty Ugly by  Tierra Whack
